### Thông tin sinh viên
- MSSV: <b>20120454</b>
- Họ tên: <b>Lê Công Đắt</b>

### Bài tập tính điểm cộng lần 2

Hãy viết chương trình giải quyết bài toán sau: Cho hệ ràng buộc gồm <b>2</b> điều kiện của bài toán QHTT <b>3</b> biến cùng hệ số của hàm mục tiêu dạng <b>max</b> - tất cả các số nhập vào đều là <b>số nguyên</b> có trị tuyệt đối không vượt quá 10.

**Yêu cầu:**
Hãy in ra bảng đơn hình xuất phát theo phương pháp big-M với số biến bổ sung là ít nhất, trong đó chọn: M=1000.

**Input:**

1 2 3 // hệ số của hàm mục tiêu <br>
1 -1 2 = 3 (dấu này có thể là >=, <= hoặc = ). <br>
3 4 -2 = 5

**Output:**

<style>
  td {
    border: none !important;
  }
</style>

<table>
  <tr>
    <td>x4</td>
    <td>-1000</td>
    <td>3</td>
    <td>1</td>
    <td>-1</td>
    <td>2</td>
    <td>1</td>
    <td>0</td>
  </tr>
  <tr>
    <td>x5</td>
    <td>-1000</td>
    <td>5</td>
    <td>3</td>
    <td>4</td>
    <td>-1</td>
    <td>0</td>
    <td>1</td>
  </tr>
  <tr>
    <td></td>
    <td></td>
    <td>-8000</td>
    <td>-4001</td>
    <td>-3002</td>
    <td>-3</td>
    <td>0</td>
    <td>0</td>
  </tr>
</table>




In [19]:
import enum

class Comparison(enum.Enum):
    LESS_EQUAL = "<="
    EQUAL = "="
    GREATER_EQUAL = ">="

a=[]
b=[]
c=[]

N = 3 # Số biến
M = 2 # Số ràng buộc
bigM = 1000 # Big-M

comparison = []

with open("input.txt", 'r') as file:
    c=list(map(int, file.readline().split()))
    for i in range(M):
        inputs = file.readline().strip().split(' ')
        inputs = list(filter(None, inputs))
        a.append(list(map(int, inputs[:N])))
        cmp = Comparison(inputs[N].strip())
        comparison.append(cmp)
        b.append(int(inputs[N+1]))

# print(c)
# print(a)
# print(comparison)
# print(b)
# print(a[0][0])

# Chuyển về dạng chính tắc
def refactor(a, comparison, M, N):
    for i in range(M):

        if b[i]<0:
            for j in range(N):
                a[i][j] = -a[i][j]
            b[i] = -b[i]
            if comparison[i] == Comparison.LESS_EQUAL:
                comparison[i] = Comparison.GREATER_EQUAL
            elif comparison[i] == Comparison.GREATER_EQUAL:
                comparison[i] = Comparison.LESS_EQUAL

        if comparison[i] == Comparison.LESS_EQUAL:
            a[i].append(1)
            for j in range(M):
                if j != i:
                    a[j].append(0)
            N += 1
        elif comparison[i] == Comparison.GREATER_EQUAL:
            a[i].append(-1)
            for j in range(M):
                if j != i:
                    a[j].append(0)
            N += 1
        
    
    return N

# print(N)
N = refactor(a, comparison, M, N)
while len(c) < N:
    c.append(0)
# print(N)
# print(a)
# print(b)
# print(c)

base_variables=[]

# Tìm cơ sở
def find_base(a, M, N):
    base_variables=[0]*M
    for i in range(M):
        for j in range(N):
            if a[i][j] == 1:
                is_ok=True
                for k in range(M):
                    if k != i and a[k][j] != 0:
                        is_ok = False
                        break
                if is_ok:
                    base_variables[i] = j + 1
                    break

    for i in range(M):
        if base_variables[i] == 0:
            base_variables[i]=N+1
            N+=1
            a[i].append(1)
            for j in range(M):
                if j != i:
                    a[j].append(0)
            c.append(-bigM)

    return N, base_variables

N, base_variables = find_base(a, M, N)
# print(base_variables)
# print(c)

delta =[0]*(N+1)

# Tính delta
def find_delta(a, c, base_variables, M, N):
    for i in range(M):
        delta[0]+=c[base_variables[i]-1]*b[i]
    for i in range(N):
        for j in range(M):
            delta[i+1]+=c[base_variables[j]-1]*a[j][i]
        delta[i+1]-=c[i]
    return delta

delta = find_delta(a, c, base_variables, M, N)
# print(delta)

def print_result(a, b, c, M, N, base_variables, delta):
    # Open the output file
    output_file = open("output.txt", "w", encoding="utf-8")
    # Calculate the width of the largest element in 'delta' list
    max_delta_width = max(len(str(i)) for i in delta)
    # Calculate the width of the largest element in 'c' list
    max_c_width = max(len(str(c[i-1])) for i in base_variables)

    # Determine the width for each column
    column_width = max(max_delta_width, max_c_width)

    # Print the rows with aligned values
    for i in range(M):
        print("x{:<{}} {:>{}} {:>{}}".format(base_variables[i], column_width, c[base_variables[i]-1], column_width, b[i], column_width), end=" ")
        output_file.write("x{:<{}} {:>{}} {:>{}}".format(base_variables[i], column_width, c[base_variables[i]-1], column_width+1, b[i], column_width+1))
        for j in range(N):
            print("{:>{}}".format(a[i][j], column_width), end=" ")
            output_file.write("{:>{}}".format(a[i][j], column_width+1))
        print("\n")
        output_file.write("\n")

    # Print the last line with aligned values
    print("{:>{}} {:>{}} ".format("", column_width, "", column_width), end=" ")
    output_file.write("{:>{}}  {:>{}} ".format("", column_width, "", column_width+1))
    for i in delta:
        print("{:>{}}".format(i, column_width), end=" ")
        output_file.write("{:>{}}".format(i, column_width+1))

print_result(a, b, c, M, N, base_variables, delta)









x4     -1000     3     1    -1     2     1     0 

x5     -1000     5     3     4    -2     0     1 

             -8000 -4001 -3002    -3     0     0 

Có thể mở rộng input (tăng số biến và ràng buộc) bằng cách sửa giá trị gán cho N, M ở phần đầu code 